In [30]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import posexplode, explode, udf, when, regexp_replace, monotonically_increasing_id, lower
from  pyspark.sql.types import StructType, StructField, StringType, MapType, ArrayType, FloatType
from googletrans import Translator
from spacy import load

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = """
    --conf spark.app.name="tratamento"
    --conf spark.driver.memory=12g
    --conf spark.executor.memory=12g
    pyspark-shell
    """

In [3]:
spark = SparkSession.builder.appName("spk").getOrCreate()

22/05/20 15:41:45 WARN Utils: Your hostname, DP6-PE07RZJL resolves to a loopback address: 127.0.1.1; using 192.168.15.200 instead (on interface wifi0)
22/05/20 15:41:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/20 15:41:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
DADOS = '../dados/train/'
SAIDA = '../dados/tratados/'

In [5]:
json_schema = StructType([
  StructField('cell_type',  MapType(StringType(),StringType()), False),
  StructField('source',  MapType(StringType(),StringType()), False)
])

In [6]:
df = spark.read.option("multiline", "true").json(DADOS, schema=json_schema)


22/05/20 15:42:07 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [8]:
df = df.limit(50)

In [9]:
df = df.withColumn('note_id', monotonically_increasing_id())

In [10]:
df_cell_type = df.select( 
    posexplode( df.cell_type ), 
    df.note_id
).withColumnRenamed('value', 'tipo')

In [11]:
df_source = df.select( 
    explode( df.source ),
    df.note_id
)

In [12]:
df_source = df_source.localCheckpoint()

In [13]:
df_source = df_source.withColumn('value',
    # regexp_replace(
        # regexp_replace(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        regexp_replace(df_source['value'] , r'<[^>]+>', ' '),
                    r'\!\[([\S\s]*?)\][\S\s]*?\)', '$1'),
                r"b\'[\s\S]*?\'", '< bytes >'),
            r'b\"[\s\S]*?\"', '< bytes >'),
        # r'<\s*table[\S\s]*?table\s*>', '< table >'),
    # r'<[\S\s]*?src\s*=[\S\s]*?>', '< image >')    
)



In [14]:
df_t = df_cell_type.join(df_source, ['key', 'note_id'], 'inner').drop('key').withColumn('value', lower('value'))

In [15]:
df_t = df_t.localCheckpoint()

In [16]:
from pyspark.sql.functions import length

In [17]:
df_t.filter( (length(df_t.value) > 10000)).count()

9

In [18]:
def traduzir(value, translator):
    if translator.detect(value).lang != 'en':
        return translator.translate(value, dest='en').text
    else:
        return value

s_traduzir = udf(lambda x: traduzir(x,  Translator() ))

In [48]:
def vetorizar(value, nlp ):
    return nlp(value).vector

s_vetorizar = udf(lambda x: vetorizar(x, load('en_core_web_md') ).tolist() )

In [49]:
# df_t = df_t.withColumn('valor', when( df_t.tipo == 'markdown', s_traduzir(df_t.value)).otherwise( df_t.value ) )

In [50]:
df_f = df_t.withColumn('vector', s_vetorizar(df_t.value) )

In [47]:
df_f.write.mode('overwrite').parquet(SAIDA)

In [51]:
spark.stop()